In [5]:
# test multiplier
import os
import numpy as np
import shutil, sys
import netCDF4 as nc

root_dir = '/home/h294liu/project/proj/5_summaCalib/5_calib_test2/BowAtBanff1/calib'
experiment_id = str(1)

archive_dir = os.path.join(root_dir, 'output_archive', experiment_id)
multp_tpl = os.path.join(root_dir, 'tpl/multipliers.tpl')
multp_txt = os.path.join(archive_dir, 'multipliers.txt')
trialParamFile_priori = os.path.join(archive_dir,'trialParams.priori.nc')
trialParamFile = os.path.join(archive_dir,'trialParams.nc')

direct_param_list = ['']  

# #### 2. Read summa param names and multiplier values
multp_names = np.loadtxt(multp_tpl, dtype='str')
param_names = [x.replace('_multp','') for x in multp_names] # derive param name by removing substring '_multp'
multp_values = np.loadtxt(multp_txt)

# update param values in trialParamFile.
with nc.Dataset(trialParamFile_priori, 'r') as src:
    with nc.Dataset(trialParamFile, 'r') as dst:

        print('Param, priori, update, multp. Fraction')
        
        for i in range(len(param_names)):
            param_name = param_names[i]
            
            
            if param_name != 'thickness':
                param_priori = src.variables[param_name][0].data               # priori param value mask array 
                param_update = dst.variables[param_name][0].data               # priori param value mask array
                multp = multp_values[i]
            
                print('%s, %f, %f, %f. %f. '%\
                      (param_name, param_priori, param_update, multp, param_update/float(param_priori)))

                if param_name == 'theta_sat':
                    for add_param in ['theta_res', 'critSoilWilting', 'critSoilTranspire', 'fieldCapacity']:
                        add_param_priori = src.variables[add_param][0].data               # priori param value mask array 
                        add_param_update = dst.variables[add_param][0].data               # priori param value mask array
                        multp = multp_values[i]
                        print('%s, %f, %f, %f. %f. '%\
                              (add_param, add_param_priori, add_param_update, multp, add_param_update/float(add_param_priori)))
                   
            else:
                for add_param in ['heightCanopyTop']:
                    add_param_priori = src.variables[add_param][0].data               # priori param value mask array 
                    add_param_update = dst.variables[add_param][0].data               # priori param value mask array
                    multp = multp_values[i]
                    print('%s, %f, %f, %f. %f. '%\
                          (add_param, add_param_priori, add_param_update, multp, add_param_update/float(add_param_priori)))

                

Param, priori, update, multp. Fraction
k_macropore, 0.001000, 0.001000, 1.000000. 1.000000. 
k_soil, 0.000001, 0.000001, 1.000000. 1.000000. 
theta_sat, 0.399000, 0.399000, 1.000000. 1.000000. 
theta_res, 0.061000, 0.061000, 1.000000. 1.000000. 
critSoilWilting, 0.155000, 0.155000, 1.000000. 1.000000. 
critSoilTranspire, 0.329000, 0.329000, 1.000000. 1.000000. 
fieldCapacity, 0.200000, 0.200000, 1.000000. 1.000000. 
aquiferBaseflowExp, 2.000000, 2.000000, 1.000000. 1.000000. 
aquiferBaseflowRate, 0.100000, 0.100000, 1.000000. 1.000000. 
qSurfScale, 50.000000, 38.864870, 0.777297. 0.777297. 
summerLAI, 3.000000, 3.000000, 1.000000. 1.000000. 
frozenPrecipMultip, 1.000000, 1.000000, 1.000000. 1.000000. 
heightCanopyBottom, 8.500000, 5.000000, 0.588235. 0.588235. 
routingGammaScale, 1487.609018, 1487.609018, 1.000000. 1.000000. 
routingGammaShape, 2.500000, 2.500000, 1.000000. 1.000000. 
Fcapil, 0.060000, 0.060000, 1.000000. 1.000000. 
heightCanopyTop, 20.000000, 16.500000, 1.000000. 0.82

In [12]:
#!/usr/bin/env python
# coding: utf-8

# #### Calculate model performance evaluation/statistical metrics.
# Author: Hongli Liu, Andy Wood.

# import module
import os, sys
import numpy as np
import datetime
import pandas as pd
import xarray as xr
import argparse, glob

def process_command_line():
    '''Parse the commandline'''
    parser = argparse.ArgumentParser(description='Script to icalculate model evaluation statistics.')
    parser.add_argument('controlFile', help='path of the active control file.')
    args = parser.parse_args()
    return(args)

def get_modified_KGE(obs,sim):    
    sd_sim=np.std(sim, ddof=1)
    sd_obs=np.std(obs, ddof=1)
    
    m_sim=np.mean(sim)
    m_obs=np.mean(obs)
    
    r=(np.corrcoef(sim,obs))[0,1]
    relvar=(float(sd_sim)/float(m_sim))/(float(sd_obs)/float(m_obs))
    bias=float(m_sim)/float(m_obs)
    
    kge=1.0-np.sqrt((r-1)**2 +(relvar-1)**2 + (bias-1)**2)
    return kge

def get_RMSE(obs,sim):
    rmse = np.sqrt(np.nanmean(np.power((sim - obs),2)))
    return rmse

def get_mean_error(obs,sim):
    bias_err = np.nanmean(sim - obs)
    abs_err = np.nanmean(np.absolute(sim - obs))
    return bias_err, abs_err

def get_month_mean_flow(obs,sim,sim_time):
    month = [dt.month for dt in sim_time]

    data = {'sim':sim, 'obs':obs, 'month':month} 
    df = pd.DataFrame(data, index = sim_time)
    
    gdf = df.groupby(['month'])
    sim_month_mean = gdf.aggregate({'sim':np.nanmean})
    obs_month_mean = gdf.aggregate({'obs':np.nanmean})
    return obs_month_mean, sim_month_mean

# Function to extract a given setting from the configuration file
def read_from_control(control_file, setting):
    
    # Open 'control_active.txt' and locate the line with setting
    with open(control_file) as ff:
        for line in ff:
            line = line.strip()
            if line.startswith(setting):
                break
    # Extract the setting's value
    substring = line.split('|',1)[1].split('#',1)[0].strip() 
    # Return this value    
    return substring
       
# Function to extract a given setting from the summa and mizuRoute manager/control files
def read_from_summa_route_control(control_file, setting):

    # Open fileManager.txt or route_control and locate the line with setting
    with open(control_file) as ff:
        for line in ff:
            line = line.strip()
            if line.startswith(setting):
                break
    # Extract the setting's value
    substring = line.split('!',1)[0].strip().split(None,1)[1].strip("'")
    # Return this value    
    return substring

# main
if __name__ == '__main__':
    
    # an example: python calc_sim_stats.py ../control_active.txt

    # ---------------------------- Preparation -------------------------------
#     # --- process command line --- 
#     # check args
#     if len(sys.argv) != 2:
#         print("Usage: %s <control_file>" % sys.argv[0])
#         sys.exit(0)
#     # otherwise continue
#     args = process_command_line()    
#     control_file = args.controlFile
    control_file = '/home/h294liu/github/summaCalib2/control_active.txt'
    
    # read paths from control_file.
    root_path = read_from_control(control_file, 'root_path')
    domain_name = read_from_control(control_file, 'domain_name')
    domain_path = os.path.join(root_path, domain_name)

    # read new hydrologic model path from control_file.
    model_dst_path = read_from_control(control_file, 'model_dst_path')
    if model_dst_path == 'default':
        model_dst_path = os.path.join(domain_path, 'model')

    # read mizuRoute setting and control files paths from control_file.
    route_settings_relpath = read_from_control(control_file, 'route_settings_relpath')
    route_settings_path = os.path.join(model_dst_path, route_settings_relpath)
    route_control = read_from_control(control_file, 'route_control')
    route_control = os.path.join(route_settings_path, route_control)

    # read calib path from control_file.
    calib_path = read_from_control(control_file, 'calib_path')
    if calib_path == 'default':
        calib_path = os.path.join(domain_path, 'calib')
    # -----------------------------------------------------------------------

    # #### 1. Read input and output arguments 
    # (input) mizuRoute output file
    output_dir = read_from_summa_route_control(route_control, '<output_dir>')
    route_outFilePrefix=read_from_summa_route_control(route_control, "<case_name>")
    
    # (input) define mizuRoute output file name based on mizuRoute source code "write_simoutput.f90".    
    route_outFileList = glob.glob(output_dir+'/'+route_outFilePrefix+'.*.nc')
    route_outFileList.sort()
    
    # (input) segment id, observations, statistics relevant configs.
    q_seg_index = int(read_from_control(control_file, 'q_seg_index')) # start from one.
    
    obs_file = read_from_control(control_file, 'obs_file')
    obs_unit = read_from_control(control_file, 'obs_unit')

    statStartDate = read_from_control(control_file, 'statStartDate') 
    statEndDate = read_from_control(control_file, 'statEndDate')

    # (input) others
    time_format='%Y-%m-%d'
    statStartDate = datetime.datetime.strptime(statStartDate,time_format)
    statEndDate = datetime.datetime.strptime(statEndDate,time_format)    

    # (output) statistical output file.
    stat_output = read_from_control(control_file, 'stat_output')
    stat_output = os.path.join(calib_path, stat_output)

    # #### 2. Calculate 
    # --- read simulated flow (cms) --- 
    simVarName = 'IRFroutedRunoff'
    simFile = os.path.join(output_dir, route_outFilePrefix+'.mizuRoute.nc') # Hard coded file name. Be careful.
#     simFile = 'run1.nc'
    f    = xr.open_dataset(simFile)
    time = f['time'].values
    sim  = f[simVarName][:,(q_seg_index-1)].values #(time, segments)
    df_sim = pd.DataFrame({'sim':sim},index = time)
    df_sim.index = pd.to_datetime(df_sim.index)

    # --- read observed flow (cfs or cms) --- 
    df_obs = pd.read_csv(obs_file, index_col='Date', na_values=["-99.0","-999.0","-9999.0"],
                         parse_dates=True, infer_datetime_format=True)  
    df_obs.columns = ['obs']
    
    # convert obs from cfs to cms
    if obs_unit == 'cfs':
        df_obs = df_obs/35.3147    
        
    # --- merge the two df based on time index--- 
    df_sim_eval = df_sim.truncate(before=statStartDate, after=statEndDate)
    df_obs_eval = df_obs.truncate(before=statStartDate, after=statEndDate)
    df_merge = pd.concat([df_obs_eval, df_sim_eval], axis=1)
    df_merge = df_merge.dropna()

    # --- calculate diagnostics --- 
    kge = get_modified_KGE(obs=df_merge['obs'].values, sim=df_merge['sim'].values)
    rmse = get_RMSE(obs=df_merge['obs'].values, sim=df_merge['sim'].values)
    # bias_err, abs_err = get_mean_error(obs=df_merge['obs'].values, sim=df_merge['sim'].values)
    # obs_month_mean, sim_month_mean = get_month_mean_flow(obs=df_merge['obs'].values, sim=df_merge['sim'].values, sim_time=sim_time)
    
#     # --- save --- 
#     f = open(stat_output, 'w+')
#     f.write('%.6f' %kge + '\t#KGE\n')
#     f.write('%.6f' %rmse + '\t#RMSE (cms)\n')
#     # f.write('%.6f' %bias_err + '\t#MBE (cms)\n')
#     # f.write('%.6f' %abs_err + '\t#MAE (cms)\n')
#     f.close()


In [28]:
# test update param (Don't want to update other soil params)
#!/usr/bin/env python
# coding: utf-8

# #### Update summa parameter values based on Ostrich generated multiplier values
# 1. Read input and output arguments from control_active.txt.
# 2. Read summa param names and multiplier values.
# 3. Update summa param values.

# import module
import os
import numpy as np
import shutil, sys
import netCDF4 as nc
import argparse

def process_command_line():
    '''Parse the commandline'''
    parser = argparse.ArgumentParser(description='Script to icalculate model evaluation statistics.')
    parser.add_argument('controlFile', help='path of the overall control file.')
    args = parser.parse_args()
    return(args)

# Function to extract a given setting from the configuration file
def read_from_control(control_file, setting):
    
    # Open 'control_active.txt' and locate the line with setting
    with open(control_file) as ff:
        for line in ff:
            line = line.strip()
            if line.startswith(setting):
                break
    # Extract the setting's value
    substring = line.split('|',1)[1].split('#',1)[0].strip() 
    # Return this value    
    return substring
       
# Function to extract a given setting from the summa and mizuRoute manager/control files
def read_from_summa_route_control(control_file, setting):

    # Open fileManager.txt or route_control and locate the line with setting
    with open(control_file) as ff:
        for line in ff:
            line = line.strip()
            if line.startswith(setting):
                break
    # Extract the setting's value
    substring = line.split('!',1)[0].strip().split(None,1)[1].strip("'")
    # Return this value    
    return substring

# main
if __name__ == '__main__':
    
    # an example: python calc_sim_stats.py ../control_active.txt

    # ---------------------------- Preparation -------------------------------
#     # --- process command line --- 
#     # check args
#     if len(sys.argv) != 2:
#         print("Usage: %s <control_file>" % sys.argv[0])
#         sys.exit(0)
#     # otherwise continue
#     args = process_command_line()    
#     control_file = args.controlFile
    control_file = '/home/h294liu/project/proj/5_summaCalib/5_calib_test2/BowAtBanff7/calib/control_active.txt'
    
    # #### 1. Preparation ####
    # read paths from control_file.
    root_path = read_from_control(control_file, 'root_path')
    domain_name = read_from_control(control_file, 'domain_name')
    domain_path = os.path.join(root_path, domain_name)

    # read new hydrologic model path from control_file.
    model_dst_path = read_from_control(control_file, 'model_dst_path')
    if model_dst_path == 'default':
        model_dst_path = os.path.join(domain_path, 'model')

    # read summa settings and fileManager paths from control_file.
    summa_settings_relpath = read_from_control(control_file, 'summa_settings_relpath')
    summa_settings_path = os.path.join(model_dst_path, summa_settings_relpath)
    summa_filemanager = read_from_control(control_file, 'summa_filemanager')
    summa_filemanager = os.path.join(summa_settings_path, summa_filemanager)

    # read calib path from control_file.
    calib_path = read_from_control(control_file, 'calib_path')
    if calib_path == 'default':
        calib_path = os.path.join(domain_path, 'calib')
    calib_tpl_path = os.path.join(calib_path, 'tpl')
    # -----------------------------------------------------------------------

    # # #### 1. Read input and output arguments
    # a list of parameters that are not calib by multiplier, but directly on param values.
    direct_param_list = ['']  

    # (input) multiplier template file generated by 4_prepare_multp_bounds.ipynb.
    multp_tpl = read_from_control(control_file, 'multp_tpl')
    multp_tpl = os.path.join(calib_tpl_path, multp_tpl)

    # (input) multiplier value file generated by Ostrich
    multp_txt = read_from_control(control_file, 'multp_value')
    multp_txt = os.path.join(calib_path, multp_txt)

    # (output) summa utilized param file
    trialParamFile = read_from_summa_route_control(summa_filemanager, 'trialParamFile')
    trialParamFile = os.path.join(summa_settings_path, trialParamFile)

    # (input) summa a priori param file 
    trialParamFile_priori = trialParamFile.split('.nc')[0] + '.priori.nc' # a priori param file
    trialParamFile_priori = os.path.join(summa_settings_path, trialParamFile_priori)

    # #### 2. Read summa param names and multiplier values
    multp_names = np.loadtxt(multp_tpl, dtype='str')
    param_names = [x.replace('_multp','') for x in multp_names] # derive param name by removing substring '_multp'
    multp_values = np.loadtxt(multp_txt)
    
#     multp_txt = '/home/h294liu/project/proj/5_summaCalib/5_calib_test2/BowAtBanff7/calib/multipliers.txt'
#     trialParamFile_priori = '/home/h294liu/project/proj/5_summaCalib/5_calib_test2/BowAtBanff7/model/settings/SUMMA/trialParams.priori.nc'
#     trialParamFile = '/home/h294liu/project/proj/5_summaCalib/5_calib_test2/BowAtBanff7/model/settings/SUMMA/trialParams.nc'
    
    # #### 3. Check summa param values
    # update param values in trialParamFile.
    print('Param, A-priori, Multiplier, New')
    with nc.Dataset(trialParamFile_priori, 'r') as src:
        with nc.Dataset(trialParamFile, 'r') as dst:

            for i in range(len(param_names)):
                param_name = param_names[i]
                if (param_name != 'thickness'):
                    param_ma_priori  = src.variables[param_name][:]
                    param_ma  = dst.variables[param_name][:]
                    print('%s,  %.5f, %.5f, %.5f'%(param_name,
                                                   src.variables[param_name][:].flat[0],
                                                   multp_values[i],
                                                   dst.variables[param_name][:].flat[0]))
                    if param_name == 'theta_sat':
                        for add_param in ['theta_res', 'critSoilWilting', 'critSoilTranspire', 'fieldCapacity']:
                            add_param_ma_priori  = src.variables[add_param][:]
                            add_param_ma  = dst.variables[add_param][:]
                            
                            fraction_old =  np.divide(add_param_ma_priori.data, param_ma_priori.data) # fraction based on priori variable values
                            fraction =  np.divide(add_param_ma.data, param_ma.data)
                            
                            print('%s,  %.5f, %.5f, %.5f'%(add_param,
                                                                 src.variables[add_param][:].flat[0],
                                                                 multp_values[i],
                                                                 dst.variables[add_param][:].flat[0]))
                            print('Old Fraction = %.5f, New Fraction = %.5f\n'%(fraction_old.flat[0], fraction.flat[0]))

                else:                    
                    tied_param_name   = 'heightCanopyBottom'
                    target_param_name = 'heightCanopyTop'

                    # update TopCanopyHeight
                    TH_param_ma_priori = src.variables[target_param_name][:]         # priori TopCanopyHeight mask array 
                    BH_param_ma_priori = src.variables[tied_param_name][:]           # priori BottomCanopyHeight mask array 
                    TH_param_ma        = dst.variables[target_param_name][:]         # updated TopCanopyHeight mask array 
                    BH_param_ma        = dst.variables[tied_param_name][:]           # updated BottomCanopyHeight mask array
                    
                    thick_priori = TH_param_ma_priori - BH_param_ma_priori
                    thick        = TH_param_ma - BH_param_ma
                    
                    print('%s,  %.5f, %.5f, %.5f'%(param_name,
                                                   thick_priori.flat[0],
                                                   multp_values[i],
                                                   thick.flat[0]))
               

Param, A-priori, Multiplier, New
k_macropore,  0.00100, 97.06843, 0.09707
k_soil,  0.00000, 0.30823, 0.00000
theta_sat,  0.39900, 0.92550, 0.36927
theta_res,  0.06100, 0.92550, 0.06100
Old Fraction = 0.15288, New Fraction = 0.16519

critSoilWilting,  0.15500, 0.92550, 0.15500
Old Fraction = 0.38847, New Fraction = 0.41974

critSoilTranspire,  0.32900, 0.92550, 0.32900
Old Fraction = 0.82456, New Fraction = 0.89094

fieldCapacity,  0.20000, 0.92550, 0.20000
Old Fraction = 0.50125, New Fraction = 0.54161

aquiferBaseflowExp,  2.00000, 4.90134, 9.80268
aquiferBaseflowRate,  0.00100, 64.61213, 0.06461
qSurfScale,  50.00000, 0.34098, 17.04911
summerLAI,  3.00000, 3.24540, 9.73620
frozenPrecipMultip,  1.00000, 1.49083, 1.49083
heightCanopyBottom,  8.50000, 0.30460, 2.58908
routingGammaScale,  1487.60902, 45.01304, 66961.80423
routingGammaShape,  2.50000, 1.12031, 2.80078
Fcapil,  0.06000, 0.18721, 0.01123
thickness,  11.50000, 0.23066, 2.65265


In [39]:
#!/usr/bin/env python
# coding: utf-8

# This code plots the parameter trace during a parameter estimation process.
# Note: This code plots the incomplete trace of samples because it reads sampels from ostOutput.txt.
# Only the parameter sets that improve the objective function in comparison with the previous parameter set are plotted.

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import os, sys, argparse, shutil, datetime
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt 

def process_command_line():
    '''Parse the commandline'''
    parser = argparse.ArgumentParser(description='Script to icalculate model evaluation statistics.')
    # Positional mandatory arguments
    parser.add_argument('controlFile', help='path of the active control file.')
    parser.add_argument("experiment_id", 
                        help="a list of calibration experiment ids to be plotted",
                        type=int, nargs='+')
    # experiment_id can take a list of integers.
    # when experiment_id = 0, it refers to files in the current calibration directory.
    
    args = parser.parse_args()
    return(args)

# Function to extract a given setting from the configuration file
def read_from_control(control_file, setting):
    
    # Open 'control_active.txt' and locate the line with setting
    with open(control_file) as ff:
        for line in ff:
            line = line.strip()
            if line.startswith(setting):
                break
    # Extract the setting's value
    substring = line.split('|',1)[1].split('#',1)[0].strip() 
    # Return this value    
    return substring

# Function to extract a given setting from the summa and mizuRoute manager/control files
def read_from_summa_route_control(control_file, setting):

    # Open fileManager.txt or route_control and locate the line with setting
    with open(control_file) as ff:
        for line in ff:
            line = line.strip()
            if line.startswith(setting):
                break
    # Extract the setting's value
    substring = line.split('!',1)[0].strip().split(None,1)[1].strip("'")
    # Return this value    
    return substring

def is_number(s):
    try:
        float(s)
        return True 
    except (ValueError,AttributeError):
        return False 
                    
def read_param_sample_from_ostOutput(ostOutput):    
    sample_ids=[]
    objs = []
    sample_values = []
    
    with open(ostOutput, 'r') as f:
        for line_number, line in enumerate(f):
            line = line.strip()
            if line and line.startswith('Ostrich Run Record'):
                line_param_name = line_number+1
                line_param_value_start = line_number+2
            elif line and line.startswith('Optimal Parameter Set'):
                line_param_value_end = line_number-2
                break
        if not 'Optimal Parameter Set' in f.read():
            line_param_value_end = line_number

    with open(ostOutput, 'r') as f:
        for line_number, line in enumerate(f):
            line = line.strip()
            if line and line_number >= line_param_value_start and line_number <= line_param_value_end:                      
                if all([is_number(s) for s in line.split()]) == True:

                    sample_ids.append(int(line.split()[0])) 
                    objs.append(float(line.split()[1]))
                    sample_values.append([float(x) for x in line.split()[2:-1]])
                
            elif line_number > line_param_value_end:
                break
    
    sample_ids=np.asarray(sample_ids)
    objs=np.asarray(objs)
    sample_values=np.asarray(sample_values)
    return  sample_ids,objs,sample_values

# Function to extract the param default values and bounds from basinParam and localParam.txt.
def read_basinParam_localParam(filename):
    param_names = []
    param_default = []
    param_min = []
    param_max =[]
    with open (filename, 'r') as f:
        for line in f:
            line=line.strip()
            if line and not line.startswith('!') and not line.startswith("'"):
                splits=line.split('|')
                if isinstance(splits[0].strip(), str):
                    param_names.append(splits[0].strip())
                    param_default.append(str_to_float(splits[1].strip()))
                    param_min.append(str_to_float(splits[2].strip()))
                    param_max.append(str_to_float(splits[3].strip()))
    return param_names, param_default, param_min, param_max

# Function to convert data from Fortran format to scientific format.
def str_to_float(data_str):
    if 'd' in data_str:
        x = data_str.split('d')[0]+'e'+data_str.split('d')[1]
        return float(x)
    else:
        return float(data_str)


# main
if __name__ == '__main__':
    
    # an example: python 1_plot_param_trace.py ../control_active.txt

#     # --- process command line --- 
#     # check args
#     if len(sys.argv) < 2:
#         print("Usage: %s <controlFile>" % sys.argv[0])
#         sys.exit(0)
#     # otherwise continue
#     args = process_command_line()    
#     control_file = args.controlFile
#     experiment_id = args.experiment_id
    
    control_file = '/home/h294liu/project/proj/5_summaCalib/5_calib_test2/BowAtBanff5/calib/control_active.txt'
    experiment_id = [1,2,3,4]

    direct_param_list = [] # a list of parameters that are directly estimated, not depending on multiplier.

    # --------------------------- Read settings -----------------------------
    # read paths from control_file.
    root_path = read_from_control(control_file, 'root_path')
    domain_name = read_from_control(control_file, 'domain_name')
    domain_path = os.path.join(root_path, domain_name)
    
    # read calib path 
    calib_path = read_from_control(control_file, 'calib_path')
    if calib_path == 'default':
        calib_path = os.path.join(domain_path, 'calib')
    analysis_path = os.path.join(calib_path, 'analysis')

    # read new hydrologic model path, settings, fileManager.txt, trialParam.nc.
    model_dst_path = read_from_control(control_file, 'model_dst_path')
    if model_dst_path == 'default':
        model_dst_path = os.path.join(domain_path, 'model')
    
    summa_settings_relpath = read_from_control(control_file, 'summa_settings_relpath')
    summa_settings_path = os.path.join(model_dst_path, summa_settings_relpath)

    summa_filemanager = read_from_control(control_file, 'summa_filemanager')
    summa_filemanager = os.path.join(summa_settings_path, summa_filemanager)

    trialParamFile = read_from_summa_route_control(summa_filemanager, 'trialParamFile')
    trialParamFile_priori = trialParamFile.split('.nc')[0] + '.priori.nc' # a priori param file

    # identify plot output path and file
    output_path = os.path.join(analysis_path, '1_plot_param_trace_partial')
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    trialParamFile_temp = os.path.join(output_path, 'trialParam.temp.nc')  # temporary trialParam file   
    
    experiment_id_str = '_'.join([str(x) for x in experiment_id])
    ofile_fig = os.path.join(output_path, 'experiment%s_trace_partial.png'%(experiment_id_str))  # output plot figure
    ofile_txt = os.path.join(output_path, 'experiment%s_summary_ostOutput.txt'%(experiment_id_str))  # output best param information
   
    # --------------------------- End Read settings -----------------------------
         
    print('Read multiplier samples.')
    # (1) read multiplier samples from OstOutput.txt 
    experiment_num = len(experiment_id)
    max_itrations = np.zeros((experiment_num))
    for i in range(experiment_num):
        
        # identify archive path, trialParamFile_priori, and ostModel.txt
        if experiment_id[i] == 0:
            archive_path = calib_path
            trialParamFile_priori = os.path.join(summa_settings_path, trialParamFile_priori)
        else:
            archive_path = os.path.join(calib_path, 'output_archive', str(experiment_id[i]))
            trialParamFile_priori = os.path.join(archive_path, trialParamFile_priori)
            
        ostOutput = os.path.join(archive_path, 'OstOutput0.txt')
        
        # read ostOutput
        sample_ids,objs,multp_values = read_param_sample_from_ostOutput(ostOutput)
        max_itrations[i] =sample_ids[-1] # record the max iteration of a calib experiment.
        
        # append 
        if i == 0:
            sample_ids_concat,objs_concat,multp_values_concat = sample_ids,objs,multp_values
        else:
            sample_ids_update = sample_ids + np.sum(max_itrations[0:i]) # calculate cumulative sampel id.
            sample_ids_concat = np.concatenate((sample_ids_concat, sample_ids_update), axis=0)
            objs_concat = np.concatenate((objs_concat, objs), axis=0)
            multp_values_concat = np.concatenate((multp_values_concat, multp_values), axis=0)
            
    # identify the best parameter sets
    best_obj = np.min(objs_concat)
    best_indices = np.argwhere(objs_concat == best_obj)
    best_iterations_str = ",".join(map(str,map(int, list(sample_ids_concat[best_indices]))))
    
    f = open(ofile_txt, "w")
    print('--- Best KGE = %.4f. Occur %d times. \nAt iterations %s. \n'%(best_obj, len(best_indices), best_iterations_str))
    f.write('Best KGE = %.4f. Occur %d times. \nAt iterations %s. \n'%(best_obj, len(best_indices), best_iterations_str))
                
    print('Read parameter names and ranges.')
    # (2a) read parameter names from summa_parameter_bounds.txt
    # param_names is a list of params corresponding to multiplier, used to update trialParam file.
    param_names = list(np.loadtxt(os.path.join(calib_path, 'summa_parameter_bounds.txt'), usecols=[0], dtype='str', delimiter=','))
    param_range = np.loadtxt(os.path.join(calib_path, 'summa_parameter_bounds.txt'), usecols=[2,3], delimiter=',')
    
    # (2b) read parameter ranges from localParam.txt and basinParam.txt.
    summa_settings_relpath = read_from_control(control_file, 'summa_settings_relpath')
    summa_setting_path = os.path.join(model_dst_path, summa_settings_relpath)

    summa_filemanager = read_from_control(control_file, 'summa_filemanager')
    summa_filemanager = os.path.join(summa_setting_path, summa_filemanager)

    basinParam = read_from_summa_route_control(summa_filemanager, 'globalGruParamFile')
    localParam = read_from_summa_route_control(summa_filemanager, 'globalHruParamFile')

    basinParam = os.path.join(summa_setting_path, basinParam)
    localParam = os.path.join(summa_setting_path, localParam)

    basin_param_names, basin_param_default, basin_param_min, basin_param_max = read_basinParam_localParam(basinParam)    
    local_param_names, local_param_default, local_param_min, local_param_max = read_basinParam_localParam(localParam)

    # (3) get a more complete list of params, used for plot. 
    output_param_names = param_names.copy() 
    
#     # add soil parameters if soil parameters are included in param_names.
#     soil_params = ['theta_res', 'critSoilWilting', 'critSoilTranspire', 'fieldCapacity', 'theta_sat']
#     if any(soil_param in param_names for soil_param in soil_params):
#         for soil_param in soil_params:
#             if not soil_param in output_param_names:
#                 output_param_names.append(soil_param)
                
    # add canopy height parameters if thickness if included in param_names.
    canopyHeigh_params = ['heightCanopyBottom', 'heightCanopyTop']
    if 'thickness' in param_names:
        output_param_names.remove('thickness')
        for canopyHeigh_param in canopyHeigh_params:
            if not canopyHeigh_param in output_param_names:
                output_param_names.append(canopyHeigh_param)
            
    # sort output_param_names.
    output_param_names.sort()

    print('Update parameter values based on multipler samples.')
    # (4) update param values based on multipler value and param priori value. 
    # and store the updated param values in array output_param_values.
    Nsample = len(sample_ids_concat)
    Nparam = len(output_param_names)
    output_param_values = np.zeros((Nsample, Nparam))
    
    with nc.Dataset(trialParamFile_priori, 'r') as src:

        # loop all multiplier samples to get corresponding parameter values       
        for iSample in range(Nsample):
            multp_iSample = multp_values_concat[iSample]           
        
            # --- copy trialParamFile_priori to be the base of trialParamFile.
            shutil.copy(trialParamFile_priori, trialParamFile_temp)

            # --- update param values in a template trialParam file and save new param values to array.
            with nc.Dataset(trialParamFile_temp, 'r+') as dst:

                for iParam in range(len(param_names)):
                    param_name = param_names[iParam]

                    # Part A: update all params except 'thickness'
                    if (param_name != 'thickness') and param_name in dst.variables.keys():  

                        # update param values
                        if not param_name in direct_param_list:# new_value = multipler * default_value
                            param_ma_priori = src.variables[param_name][:]               # priori param value mask array 
                            param_value     = param_ma_priori.data * multp_iSample[iParam]     # update param value mask array
                            dst.variables[param_name][:] = np.ma.array(param_value, \
                                                                       mask=np.ma.getmask(param_ma_priori), \
                                                                       fill_value=param_ma_priori.get_fill_value())
                        elif param_name in direct_param_list: # new_value = Ostrich value
                            param_ma_priori = src.variables[param_name][:]                          # priori param value mask array 
                            param_value     = np.ones_like(param_ma_priori.data) * multp_iSample[iParam]  # update param value mask array
                            dst.variables[param_name][:] = np.ma.array(param_value, \
                                                                       mask=np.ma.getmask(param_ma_priori),  \
                                                                       fill_value=param_ma_priori.get_fill_value())                   

#                         # if param is 'theta_sat', update other four soil variables using a priori param value fractions.
#                         if param_name == 'theta_sat':
#                             param_ma_priori  = src.variables[param_name][:]
#                             param_ma = dst.variables[param_name][:]

#                             for add_param in ['theta_res', 'critSoilWilting', 'critSoilTranspire', 'fieldCapacity']:
#                                 add_param_ma_priori  = src.variables[add_param][:]
#                                 fraction =  np.divide(add_param_ma_priori.data, param_ma_priori.data) # fraction based on priori variable values
#                                 add_param_value = param_ma.data * fraction
#                                 dst.variables[add_param][:]= np.ma.array(add_param_value,                                                                          mask=np.ma.getmask(add_param_ma_priori),                                                                          fill_value=add_param_ma_priori.get_fill_value())

                # After updating all parameters, update 'thickness' if it exists. 
                # Actually, use 'thickness' to calculate TopCanopyHeight.
                if 'thickness' in param_names:
                    tied_param_name   = 'heightCanopyBottom'
                    target_param_name = 'heightCanopyTop'

                    # update TopCanopyHeight
                    TH_param_ma_priori = src.variables[target_param_name][:]         # priori TopCanopyHeight mask array 
                    BH_param_ma_priori = src.variables[tied_param_name][:]           # priori BottomCanopyHeight mask array 
                    BH_param_ma        = dst.variables[tied_param_name][:]           # updated BottomCanopyHeight mask array
                    param_value        = BH_param_ma.data + \
                    (TH_param_ma_priori.data-BH_param_ma_priori.data)*multp_iSample[iParam]    # updated TopCanopyHeight values
                    dst.variables[target_param_name][:] = np.ma.array(param_value, \
                                                                      mask=np.ma.getmask(TH_param_ma_priori),  \
                                                                      fill_value=TH_param_ma_priori.get_fill_value())
            
                #  Part B: Extract output parameter values for the first GRU/HRU.
                # Be careful. Hard coded!
                for jParam in range(Nparam):
                    param_name = output_param_names[jParam]
                    param_ma = dst.variables[param_name][:]
                    output_param_values[iSample, jParam]=param_ma.data.flat[0]

    print('Plot parameter traces.')
    # (5) plot objective function and parameter trace.
    col_num = 4        
    row_num = int(np.ceil(Nparam/float(col_num)))
    
    fig, ax = plt.subplots(row_num,col_num, figsize=(3.0*col_num, 3.0*0.75*row_num), constrained_layout=True)
#     fig.suptitle('Calibration Experiment %d'%(experiment_id), fontsize='medium', fontweight='semibold')
    dpi_value=80

    for i in range(row_num):
        for j in range(col_num):
            
            subplot_count = i*col_num + j
            param_index = subplot_count - 1
            
            if subplot_count <= Nparam: 
                # plot obj function 
                if i==0 and j==0: 
                    ax[i,j].plot(sample_ids_concat, objs_concat,color='black',marker='o',
                                 linewidth=0.75,markersize=1.0, label='Sample')
                    fig_count = i*col_num + j
                    title_str = '('+chr(ord('a') + subplot_count) +') ' + 'Objective function'   
                    y_label = '-KGE'
                    
                    # plot best obj
                    ax[i,j].plot(sample_ids_concat[0], objs_concat[0], 
                                 'D', markerfacecolor="none", markeredgecolor="blue", markersize=4, label='Initial');
                    ax[i,j].plot(sample_ids_concat[best_indices], objs_concat[best_indices], 
                                 's', markerfacecolor="none", markeredgecolor="red", markersize=4, label='Best');

                # plot parameters
                else: 
                    param_name = output_param_names[param_index]
                    ax[i,j].plot(sample_ids_concat, output_param_values[:, param_index],color='black',marker='o',
                                 linewidth=0.75,markersize=1.0,label='Sample')
                    title_str = '('+chr(ord('a') + subplot_count) +') ' + param_name
                    y_label = 'Value' 
                
                    # plot the initial and best points 
                    ax[i,j].plot(sample_ids_concat[0], output_param_values[0, param_index], 
                                 'D', markerfacecolor="none", markeredgecolor="blue", markersize=4, label='Initial');
                    
                    best_param_values = output_param_values[best_indices, param_index]
                    ax[i,j].plot(sample_ids_concat[best_indices], best_param_values, 
                                 's', markerfacecolor="none", markeredgecolor="red", markersize=4, label='Best');
                    
                    #  if there are multiple best param sets, and all best param values are the same.
                    if len(best_indices)>1 and np.all(best_param_values == best_param_values[0]):
                        print('--- %s = %.6f.'%(param_name, best_param_values[-1]))   
                        f.write('%s = %.6f.\n'%(param_name, best_param_values[-1]))
                    else: # best param values are different.
                        best_param_values_str = ",".join(map(str,map(float, list(best_param_values))))
                        print('--- %s = %s.'%(param_name, best_param_values_str))   
                        f.write('%s = %s.\n'%(param_name, best_param_values_str))

                # ylimit, axis, label, title, legend
                if not (i==0 and j==0):
                    if param_name in param_names:
                        param_min = param_range[param_names.index(param_name)][0]
                        param_max = param_range[param_names.index(param_name)][1]
                    elif param_name in local_param_names:
                        param_min = local_param_min[local_param_names.index(param_name)]
                        param_max = local_param_max[local_param_names.index(param_name)]
                    elif param_name in basin_param_names:
                        param_min = basin_param_min[basin_param_names.index(param_name)]
                        param_max = basin_param_max[basin_param_names.index(param_name)]            
                    ax[i,j].set_ylim([param_min, param_max])

                ax[i,j].set_title(title_str, fontsize='small', fontweight='semibold')
                ax[i,j].set_xlabel('Iterations', fontsize='small')
                ax[i,j].set_ylabel(y_label, fontsize='small')
                
                if (i==0 and j==0):
                    ax[i,j].legend(labelspacing=0.4, loc='best', fontsize='small')
            
            # blank subplot           
            else: 
                ax[i,j].axis('off')

    plt.rc('xtick',labelsize='small')
    plt.rc('ytick',labelsize='small')                
    fig.savefig(ofile_fig, dpi=dpi_value)
    plt.close(fig)  
    if os.path.exists(trialParamFile_temp): 
        os.remove(trialParamFile_temp)         
    f.close()


Read multiplier samples.
--- Best KGE = -0.7437. Occur 14 times. 
At iterations 855,874,885,930,941,942,943,948,963,983,987,989,997,1000. 

Read parameter names and ranges.
Update parameter values based on multipler samples.
Plot parameter traces.
--- Fcapil = 0.010021.
--- aquiferBaseflowExp = 9.932204.
--- aquiferBaseflowRate = 0.000824.
--- frozenPrecipMultip = 1.497080.
--- heightCanopyBottom = 0.008354.
--- heightCanopyTop = 0.188367.
--- k_macropore = 0.099411.
--- k_soil = 0.000002.
--- qSurfScale = 6.950840.
--- routingGammaScale = 486.080858.
--- routingGammaShape = 2.999425.
--- summerLAI = 6.608613,6.1276470000000005,6.118059000000001,7.288197,9.440046,6.241638,6.69039,6.671007,5.261814,6.168291,7.747479,8.438865,9.289683,9.289683.
--- theta_sat = 0.509031.


In [37]:
sample_ids_concat.astype(int)

array([   1,    5,   10,   15,   20,   25,   26,   28,   37,   47,   48,
         69,   74,   78,   82,   85,   88,   91,   92,   97,  101,  102,
        103,  105,  107,  116,  120,  130,  138,  149,  152,  153,  154,
        162,  164,  165,  166,  171,  178,  180,  181,  187,  191,  193,
        195,  200,  201,  212,  226,  232,  245,  248,  251,  271,  291,
        296,  297,  301,  302,  330,  349,  352,  353,  354,  362,  365,
        366,  391,  396,  400,  401,  442,  446,  447,  460,  490,  491,
        492,  495,  499,  504,  536,  545,  547,  557,  566,  568,  569,
        574,  578,  583,  585,  609,  616,  630,  632,  641,  642,  643,
        648,  659,  663,  671,  683,  687,  689,  691,  692,  697,  698,
        700,  701,  746,  768,  835,  855,  874,  885,  930,  941,  942,
        943,  948,  963,  983,  987,  989,  997, 1000])

In [38]:
objs_concat

array([-0.183549, -0.22905 , -0.27551 , -0.299083, -0.34631 , -0.387305,
       -0.398824, -0.414626, -0.41486 , -0.458983, -0.460478, -0.461119,
       -0.463615, -0.473991, -0.475042, -0.484068, -0.505307, -0.505307,
       -0.511915, -0.519529, -0.519632, -0.519632, -0.519891, -0.522122,
       -0.522431, -0.522581, -0.522789, -0.522789, -0.522814, -0.522815,
       -0.522815, -0.52289 , -0.52289 , -0.5229  , -0.523204, -0.536548,
       -0.536548, -0.565196, -0.588726, -0.588806, -0.588918, -0.590692,
       -0.590692, -0.596625, -0.596632, -0.596632, -0.596632, -0.604652,
       -0.608417, -0.616079, -0.630786, -0.639665, -0.642941, -0.643559,
       -0.643559, -0.646558, -0.648423, -0.649956, -0.649956, -0.649956,
       -0.650515, -0.650515, -0.650791, -0.650791, -0.650852, -0.651841,
       -0.651841, -0.651841, -0.65247 , -0.65247 , -0.65247 , -0.672007,
       -0.706278, -0.718136, -0.722315, -0.724454, -0.725107, -0.730855,
       -0.734633, -0.736209, -0.736317, -0.736529, 

In [34]:
max_itrations

array([300., 200., 300., 300.])

In [35]:
len(sample_ids_concat), len(objs_concat)

(101, 101)